In [1]:
import geopandas as gpd
import pandas as pd
import os
from itertools import chain

In [2]:
path = r'T:\Data\LTD Data'

In [3]:
pathlist = ['2013 Fall_Oct2013_Routes and Stops\Fall 2013 LTD Routes',
            '2014 Fall Oct2014_Routes and Stops',
            '2015 Fall LTD Routes and Stops',
            '2016 Fall LTD Routes and Stops',
            '2016 Winter LTD Routes and Stops',
            '2018 Fall LTD Routes and Stops',
            '2019 Fall LTD Routes and Stops',
            '2020 Winter LTD Routes and Stops',
            '2020 Winter LTD Routes and Stops', 
            '2022 Fall LTD Routes and Stops']

In [4]:
lyrlist = ['Fall 2013 LTD Routes', 'Fall 2014 LTD routes', 'Fall 2015 LTD routes',
           'Fall 2016 LTD Routes', 'Winter 2016 LTD routes', 'Fall_2018_Routes',
           'Fall_2019_Routes', 'Winter_2020_Routes', 'Winter_2020_Routes', 'Fall_2022_Routes']

In [5]:
frames = []
for i in range(0, len(pathlist)):
    routes = gpd.read_file(os.path.join(path, pathlist[i]), layer=lyrlist[i])
    routes = routes.to_crs(epsg=2992)
    routes.columns = list(map(lambda x: x.lower(), routes.columns))
    gdf = routes[['route', 'geometry']].dissolve('route')
    gdf['route'] = gdf.index
    gdf.reset_index(drop=True, inplace=True)
    frames.append(gdf)

In [6]:
routes = pd.concat(frames, ignore_index=True)

In [7]:
gdf = routes.dissolve(by='route', aggfunc='last')
gdf['route'] = gdf.index
gdf.reset_index(drop=True, inplace=True)

In [8]:
gdf.route.unique()

array(['1', '11', '12', '13', '17', '18', '19', '24', '27', '28', '32',
       '33', '36', '40', '41', '43', '51', '52', '55', '66', '67', '73',
       '76', '78', '79x', '81', '82', '85', '91', '92', '93', '95', '96',
       '98', 'EmX', 'EmX Blue', 'EmX Green'], dtype=object)

In [10]:
outpath = r'T:\MPO\Bike&Ped\BikeCounting\StoryMap\BikeOnBuses\Output'

In [11]:
gdf[~gdf.route.isin(['EmX Blue', 'EmX Green'])].to_file(os.path.join(outpath, 'LTD_Routes.shp'))

In [13]:
len(gdf.route.values)

37

In [14]:
lyrlist1 = ['Fall 2013 LTD Stops', 'fall 2014 stops + routes', 'Fall 2015 LTD Stops+routes',
           'Fall 2016 LTD stops', 'Winter 2016 LTD Stops+Routes', 'Fall_2018_Stops', 
           'Fall_2019_Stops', 'Winter_2020_Stops', 'Winter_2020_Stops', 'Fall_2022_Stops']

In [15]:
frames = []
for i in range(0, len(pathlist)):
    #print(range(2013, 2023)[i])
    if i == 0:
        stops = gpd.read_file(os.path.join(path, '2013 Fall_Oct2013_Routes and Stops\Fall 2013 LTD Stops'), 
                              layer=lyrlist1[i])
        stops = stops.to_crs(epsg=2992)
    else:
        if i not in range(5, 9):
            stops = gpd.read_file(os.path.join(path, pathlist[i]), layer=lyrlist1[i])
            stops = stops.to_crs(epsg=2992)
    stops.columns = list(map(lambda x: x.lower(), stops.columns))
    #print(stops.routes.unique())
    frames.append(stops[['stop_name', 'routes', 'geometry']])

In [16]:
stops = pd.concat(frames, ignore_index=True)

In [17]:
stops = stops.drop_duplicates(subset=['stop_name', 'routes'], keep='last')

In [18]:
stops = stops[(stops.routes!=0) & (~stops.routes.isnull())]

In [19]:
len(stops.stop_name.unique())

1505

In [20]:
g = stops.groupby('stop_name')

In [21]:
df = g.apply(lambda x: ', '.join(x.routes.astype(str).unique())).to_frame(name='routes').reset_index()

In [22]:
ndf = df.reindex(df.index.repeat(df.routes.apply(lambda x: len(x.split(', ')))))

In [23]:
ndf.loc[:, 'route'] = list(chain.from_iterable(df.routes.apply(lambda x: x.split(', '))))

In [25]:
ndf = ndf.merge(stops[['stop_name', 'geometry']], on='stop_name')

In [26]:
ndf = ndf.drop_duplicates(subset=['stop_name', 'routes', 'route'], keep='last')

In [31]:
gdf = gpd.GeoDataFrame(ndf, geometry='geometry')

In [33]:
gdf.to_file(os.path.join(outpath, 'LTD_Stops.shp'))

In [34]:
len(gdf.stop_name.unique())

1505